## Airline Passengers Prediction-Time Series Stacked LSTM with Memory Between Batches

#### LSTM for international airline passengers problem with regression framing

In [1]:
# importing required libraries

import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import math

import tensorflow as tf
tf.config.list_physical_devices('GPU')

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import LSTM

from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error


# fix random seed for reproducibility
np.random.seed(7)

print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))

Num GPUs Available:  1


In [2]:
# loading the dataset

dataset_path = 'C:/DevelopmentPlayground/Datasets/international-airline-passengers.csv'

dataframe = pd.read_csv(dataset_path, engine='python', skipfooter=3)

In [3]:
dataframe.head()

,Month,Passengers
0,1949-01,112
1,1949-02,118
2,1949-03,132
3,1949-04,129
4,1949-05,121


In [4]:
dataframe.tail()

,Month,Passengers
138,1960-07,622
139,1960-08,606
140,1960-09,508
141,1960-10,461
142,1960-11,390


In [5]:
# Reshaping the data either using array.reshape(-1, 1) if the data has a single feature
# or  array.reshape(1, -1) if it contains a single sample
dataset = dataframe["Passengers"]

dataset = np.array(dataset).reshape(-1, 1)

dataset[:5] # selecting ist 5 rows

array([[112],
       [118],
       [132],
       [129],
       [121]], dtype=int64)

In [6]:
# converting int to float

dataset = dataset.astype('float32')

dataset[:5] # selecting ist 5 rows

array([[112.],
       [118.],
       [132.],
       [129.],
       [121.]], dtype=float32)

In [7]:
# normalize the dataset - in ranage of (0, 1)

scaler = MinMaxScaler(feature_range=(0, 1))
dataset = scaler.fit_transform(dataset)

In [8]:
# Splitting the dataset ito train and test set

train_size = int(len(dataset) * 0.67)
test_size = len(dataset) - train_size

train, test = dataset[0:train_size, :], dataset[train_size: len(dataset), :]

In [9]:
print('train :', len(train))
print('test :', len(test))

train : 95
test : 48


In [10]:
# convert an array of values into a dataset matrix
def create_dataset(dataset, look_back=1):
    dataX, dataY = [], []
    for i in range(len(dataset) - look_back - 1):
        a = dataset[i: (i + look_back), 0]
        dataX.append(a)
        dataY.append(dataset[i + look_back, 0])
        
    return np.array(dataX), np.array(dataY)

In [11]:
# reshape into X=t and y= t+1

look_back = 3

train_X, train_Y = create_dataset(train, look_back)
test_X, test_Y = create_dataset(test, look_back)

In [12]:
# checking Ist 5 rows
print('train_X :', train_X[0:5])

train_X : [[0.01544401 0.02702703 0.05405405]
 [0.02702703 0.05405405 0.04826255]
 [0.05405405 0.04826255 0.03281853]
 [0.04826255 0.03281853 0.05984557]
 [0.03281853 0.05984557 0.08494207]]


In [13]:
# checking Ist 5 rows
print('train_Y :', train_Y[0:5])

train_Y : [0.04826255 0.03281853 0.05984557 0.08494207 0.08494207]


In [14]:
# checking Ist 5 rows
print('test_X :', test_X[0:5])

test_X : [[0.3899614  0.4073359  0.3803089 ]
 [0.4073359  0.3803089  0.48648646]
 [0.3803089  0.48648646 0.47104248]
 [0.48648646 0.47104248 0.484556  ]
 [0.47104248 0.484556   0.6138996 ]]


In [15]:
# checking Ist 5 rows
print('test_Y :', test_Y[0:5])

test_Y : [0.48648646 0.47104248 0.484556   0.6138996  0.6969112 ]


In [16]:
train_X.shape

(91, 3)

In [17]:
# The LSTM network expects the input data (X) to be provided with a specific array structure in
# the form of: [samples, time steps, features].

# reshape input to be [samples, time steps, features]
# reshape from [samples, timesteps] into [samples, timesteps, features]


n_features = 1
train_X = np.reshape(train_X, (train_X.shape[0], train_X.shape[1], n_features))
test_X = np.reshape(test_X, (test_X.shape[0], test_X.shape[1], n_features))

In [18]:
print('train_X :', train_X[0:5]) # checking Ist 5 rows

train_X : [[[0.01544401]
  [0.02702703]
  [0.05405405]]

 [[0.02702703]
  [0.05405405]
  [0.04826255]]

 [[0.05405405]
  [0.04826255]
  [0.03281853]]

 [[0.04826255]
  [0.03281853]
  [0.05984557]]

 [[0.03281853]
  [0.05984557]
  [0.08494207]]]


In [19]:
print('test_X :', test_X[0:5]) # checking Ist 5 rows

test_X : [[[0.3899614 ]
  [0.4073359 ]
  [0.3803089 ]]

 [[0.4073359 ]
  [0.3803089 ]
  [0.48648646]]

 [[0.3803089 ]
  [0.48648646]
  [0.47104248]]

 [[0.48648646]
  [0.47104248]
  [0.484556  ]]

 [[0.47104248]
  [0.484556  ]
  [0.6138996 ]]]


In [20]:
# create and fit the one to one LSTM network

# LSTM network has a visible layer with 1 input i.e. input_dim=look_back, 
# a hidden layer with 4 LSTM blocks or Neurons 
# with default 'tanh' activation and 'sigmoid' being default recurrent_activation
# a fully connected layer (Dense) with single neuron.
# batch_input_shape() = This is useful for specifying a fixed batch size (e.g. with stateful RNNs)

# batch_input_shape() receive a tuple (round bracket with 3 values): (batchsize, timestep, input_dim)
# I translate it to: (number_of_rows_to_process_each_loop, the_time_interval_for_next_move(e.g. per day, per month), column)

batch_size = 1
model = Sequential()
model.add(LSTM(units=4, batch_input_shape=(batch_size, look_back, 1), return_sequences=True, stateful=True, activation='tanh', recurrent_activation='sigmoid')) 
model.add(Dense(1))
model.compile(loss='mean_squared_error', optimizer='rmsprop')



In [21]:
# fit the model
import datetime
start = datetime.datetime.now()

for i in range(100):
    print('i now : ', i)
    model.fit(train_X, train_Y, epochs=10, batch_size=batch_size, verbose=1, shuffle=False)
    model.reset_states()
    
stop = datetime.datetime.now()
print("Total time taken : ", str(stop - start))

Epoch 1/10
91/91 [==============================] - 5s 6ms/step - loss: 0.0054
Epoch 2/10
91/91 [==============================] - 1s 6ms/step - loss: 0.0071
Epoch 3/10
91/91 [==============================] - 1s 6ms/step - loss: 0.0071
Epoch 4/10
91/91 [==============================] - 1s 6ms/step - loss: 0.0070
Epoch 5/10
91/91 [==============================] - 1s 6ms/step - loss: 0.0070
Epoch 6/10
91/91 [==============================] - 1s 6ms/step - loss: 0.0069
Epoch 7/10
91/91 [==============================] - 1s 6ms/step - loss: 0.0069
Epoch 8/10
91/91 [==============================] - 1s 6ms/step - loss: 0.0068
Epoch 9/10
91/91 [==============================] - 1s 6ms/step - loss: 0.0068
Epoch 10/10
91/91 [==============================] - 1s 6ms/step - loss: 0.0067
Epoch 1/10
91/91 [==============================] - 1s 6ms/step - loss: 0.0055
Epoch 2/10
91/91 [==============================] - 1s 6ms/step - loss: 0.0067
Epoch 3/10
91/91 [==============================] -

91/91 [==============================] - 1s 6ms/step - loss: 0.0045
Epoch 5/10
91/91 [==============================] - 1s 6ms/step - loss: 0.0045
Epoch 6/10
91/91 [==============================] - 1s 6ms/step - loss: 0.0045
Epoch 7/10
91/91 [==============================] - 1s 6ms/step - loss: 0.0044
Epoch 8/10
91/91 [==============================] - 1s 6ms/step - loss: 0.0044
Epoch 9/10
91/91 [==============================] - 1s 6ms/step - loss: 0.0044
Epoch 10/10
91/91 [==============================] - 1s 6ms/step - loss: 0.0044
Epoch 1/10
91/91 [==============================] - 1s 6ms/step - loss: 0.0041
Epoch 2/10
91/91 [==============================] - 1s 6ms/step - loss: 0.0044
Epoch 3/10
91/91 [==============================] - 1s 6ms/step - loss: 0.0044
Epoch 4/10
91/91 [==============================] - 1s 6ms/step - loss: 0.0043
Epoch 5/10
91/91 [==============================] - 0s 5ms/step - loss: 0.0043
Epoch 6/10
91/91 [==============================] - 1s 6ms/ste

91/91 [==============================] - 1s 6ms/step - loss: 0.0042
Epoch 8/10
91/91 [==============================] - 1s 6ms/step - loss: 0.0042
Epoch 9/10
91/91 [==============================] - 1s 6ms/step - loss: 0.0043
Epoch 10/10
91/91 [==============================] - 1s 6ms/step - loss: 0.0043
Epoch 1/10
91/91 [==============================] - 1s 6ms/step - loss: 0.0038
Epoch 2/10
91/91 [==============================] - 1s 6ms/step - loss: 0.0043
Epoch 3/10
91/91 [==============================] - 1s 6ms/step - loss: 0.0043
Epoch 4/10
91/91 [==============================] - 1s 6ms/step - loss: 0.0044
Epoch 5/10
91/91 [==============================] - 1s 6ms/step - loss: 0.0044
Epoch 6/10
91/91 [==============================] - 1s 6ms/step - loss: 0.0044
Epoch 7/10
91/91 [==============================] - 1s 6ms/step - loss: 0.0044
Epoch 8/10
91/91 [==============================] - 1s 6ms/step - loss: 0.0044
Epoch 9/10
91/91 [==============================] - 1s 6ms/ste

91/91 [==============================] - 1s 6ms/step - loss: 0.0043
Epoch 1/10
91/91 [==============================] - 1s 6ms/step - loss: 0.0045
Epoch 2/10
91/91 [==============================] - 1s 6ms/step - loss: 0.0043
Epoch 3/10
91/91 [==============================] - 1s 6ms/step - loss: 0.0043
Epoch 4/10
91/91 [==============================] - 1s 6ms/step - loss: 0.0043
Epoch 5/10
91/91 [==============================] - 1s 6ms/step - loss: 0.0043
Epoch 6/10
91/91 [==============================] - 1s 6ms/step - loss: 0.0043
Epoch 7/10
91/91 [==============================] - 1s 6ms/step - loss: 0.0043
Epoch 8/10
91/91 [==============================] - 1s 6ms/step - loss: 0.0043
Epoch 9/10
91/91 [==============================] - 1s 6ms/step - loss: 0.0043
Epoch 10/10
91/91 [==============================] - 1s 6ms/step - loss: 0.0043
Epoch 1/10
91/91 [==============================] - 1s 6ms/step - loss: 0.0045
Epoch 2/10
91/91 [==============================] - 1s 6ms/ste

91/91 [==============================] - 1s 6ms/step - loss: 0.0039
Epoch 4/10
91/91 [==============================] - 1s 6ms/step - loss: 0.0039
Epoch 5/10
91/91 [==============================] - 1s 6ms/step - loss: 0.0039
Epoch 6/10
91/91 [==============================] - 1s 6ms/step - loss: 0.0039
Epoch 7/10
91/91 [==============================] - 1s 6ms/step - loss: 0.0039
Epoch 8/10
91/91 [==============================] - 1s 6ms/step - loss: 0.0039
Epoch 9/10
91/91 [==============================] - 1s 6ms/step - loss: 0.0039
Epoch 10/10
91/91 [==============================] - 1s 6ms/step - loss: 0.0039
Epoch 1/10
91/91 [==============================] - 1s 6ms/step - loss: 0.0041
Epoch 2/10
91/91 [==============================] - 1s 6ms/step - loss: 0.0039
Epoch 3/10
91/91 [==============================] - 1s 6ms/step - loss: 0.0039
Epoch 4/10
91/91 [==============================] - 1s 6ms/step - loss: 0.0039
Epoch 5/10
91/91 [==============================] - 1s 6ms/ste

91/91 [==============================] - 1s 6ms/step - loss: 0.0034
Epoch 7/10
91/91 [==============================] - 1s 6ms/step - loss: 0.0034
Epoch 8/10
91/91 [==============================] - 1s 7ms/step - loss: 0.0034
Epoch 9/10
91/91 [==============================] - 1s 6ms/step - loss: 0.0034
Epoch 10/10
91/91 [==============================] - 1s 6ms/step - loss: 0.0034
Epoch 1/10
91/91 [==============================] - 1s 6ms/step - loss: 0.0040
Epoch 2/10
91/91 [==============================] - 1s 6ms/step - loss: 0.0034
Epoch 3/10
91/91 [==============================] - 1s 6ms/step - loss: 0.0034
Epoch 4/10
91/91 [==============================] - 1s 6ms/step - loss: 0.0034
Epoch 5/10
91/91 [==============================] - 1s 6ms/step - loss: 0.0034
Epoch 6/10
91/91 [==============================] - 1s 6ms/step - loss: 0.0034
Epoch 7/10
91/91 [==============================] - 1s 6ms/step - loss: 0.0034
Epoch 8/10
91/91 [==============================] - 1s 6ms/ste

91/91 [==============================] - 0s 5ms/step - loss: 0.0040
Epoch 10/10
91/91 [==============================] - 0s 5ms/step - loss: 0.0041
Epoch 1/10
91/91 [==============================] - 1s 6ms/step - loss: 0.0051
Epoch 2/10
91/91 [==============================] - 1s 6ms/step - loss: 0.0042
Epoch 3/10
91/91 [==============================] - 1s 6ms/step - loss: 0.0043
Epoch 4/10
91/91 [==============================] - 1s 6ms/step - loss: 0.0043
Epoch 5/10
91/91 [==============================] - 1s 6ms/step - loss: 0.0044
Epoch 6/10
91/91 [==============================] - 1s 6ms/step - loss: 0.0044
Epoch 7/10
91/91 [==============================] - 1s 6ms/step - loss: 0.0044
Epoch 8/10
91/91 [==============================] - 1s 6ms/step - loss: 0.0045
Epoch 9/10
91/91 [==============================] - 1s 6ms/step - loss: 0.0045
Epoch 10/10
91/91 [==============================] - 1s 6ms/step - loss: 0.0045
Epoch 1/10
91/91 [==============================] - 1s 6ms/st

91/91 [==============================] - 1s 6ms/step - loss: 0.0065
Epoch 3/10
91/91 [==============================] - 1s 5ms/step - loss: 0.0064
Epoch 4/10
91/91 [==============================] - 0s 5ms/step - loss: 0.0064
Epoch 5/10
91/91 [==============================] - 1s 6ms/step - loss: 0.0063
Epoch 6/10
91/91 [==============================] - 1s 6ms/step - loss: 0.0062
Epoch 7/10
91/91 [==============================] - 0s 5ms/step - loss: 0.0061
Epoch 8/10
91/91 [==============================] - 1s 6ms/step - loss: 0.0061
Epoch 9/10
91/91 [==============================] - 0s 5ms/step - loss: 0.0060
Epoch 10/10
91/91 [==============================] - 1s 6ms/step - loss: 0.0060
Epoch 1/10
91/91 [==============================] - 1s 6ms/step - loss: 0.0071
Epoch 2/10
91/91 [==============================] - 1s 6ms/step - loss: 0.0058
Epoch 3/10
91/91 [==============================] - 1s 5ms/step - loss: 0.0058
Epoch 4/10
91/91 [==============================] - 1s 6ms/ste

91/91 [==============================] - 1s 6ms/step - loss: 0.0047
Epoch 6/10
91/91 [==============================] - 1s 6ms/step - loss: 0.0047
Epoch 7/10
91/91 [==============================] - 0s 5ms/step - loss: 0.0047
Epoch 8/10
91/91 [==============================] - 0s 5ms/step - loss: 0.0047
Epoch 9/10
91/91 [==============================] - 1s 6ms/step - loss: 0.0047
Epoch 10/10
91/91 [==============================] - 1s 6ms/step - loss: 0.0047
Epoch 1/10
91/91 [==============================] - 1s 6ms/step - loss: 0.0053
Epoch 2/10
91/91 [==============================] - 1s 6ms/step - loss: 0.0047
Epoch 3/10
91/91 [==============================] - 1s 6ms/step - loss: 0.0047
Epoch 4/10
91/91 [==============================] - 1s 6ms/step - loss: 0.0047
Epoch 5/10
91/91 [==============================] - 1s 6ms/step - loss: 0.0047
Epoch 6/10
91/91 [==============================] - 1s 6ms/step - loss: 0.0047
Epoch 7/10
91/91 [==============================] - 1s 6ms/ste

91/91 [==============================] - 1s 6ms/step - loss: 0.0046
Epoch 9/10
91/91 [==============================] - 1s 6ms/step - loss: 0.0046
Epoch 10/10
91/91 [==============================] - 1s 6ms/step - loss: 0.0046
Epoch 1/10
91/91 [==============================] - 1s 6ms/step - loss: 0.0048
Epoch 2/10
91/91 [==============================] - 1s 6ms/step - loss: 0.0046
Epoch 3/10
91/91 [==============================] - 1s 6ms/step - loss: 0.0046
Epoch 4/10
91/91 [==============================] - 1s 6ms/step - loss: 0.0046
Epoch 5/10
91/91 [==============================] - 1s 6ms/step - loss: 0.0046
Epoch 6/10
91/91 [==============================] - 1s 6ms/step - loss: 0.0046
Epoch 7/10
91/91 [==============================] - 1s 6ms/step - loss: 0.0046
Epoch 8/10
91/91 [==============================] - 1s 6ms/step - loss: 0.0046
Epoch 9/10
91/91 [==============================] - 1s 6ms/step - loss: 0.0046
Epoch 10/10
91/91 [==============================] - 1s 6ms/st

In [22]:
# make predictions

train_predict = model.predict(train_X, batch_size=batch_size)
model.reset_states()
test_predict = model.predict(test_X, batch_size=batch_size)

In [23]:
print(train_predict[0:1])
train_predict = scaler.inverse_transform(train_predict)
print(train_predict[0:1])

[[[0.3808881 ]
  [0.149396  ]
  [0.08260709]]]


ValueError: Found array with dim 3. Estimator expected <= 2.

In [ ]:
# invert the prediction, because we have scaled the data using MinMaxScaler

train_predict = scaler.inverse_transform(train_predict)

train_Y = scaler.inverse_transform([train_Y])

test_predict = scaler.inverse_transform(test_predict)

test_Y = scaler.inverse_transform([test_Y])


In [ ]:
# calculate the root mean squared error

train_score = math.sqrt(mean_squared_error(train_Y[0], train_predict[:, 0]))

print('Train Score : %.2f RMSE' %(train_score))

In [ ]:
test_score = math.sqrt(mean_squared_error(test_Y[0], test_predict[:, 0]))

print('Test Score : %.2f RMSE' %(test_score))

In [ ]:
# shift train predictions for plotting

trainPredictPlot = np.empty_like(dataset)
trainPredictPlot[:, :] = np.nan
trainPredictPlot[look_back:len(train_predict) + look_back, :] = train_predict

In [ ]:
# shift test predictions for plotting

testPredictPlot = np.empty_like(dataset)
testPredictPlot[:, :] = np.nan
testPredictPlot[len(train_predict) + (look_back * 2) + 1:len(dataset) - 1, :] = test_predict

In [ ]:
# plot baseline and predictions

plt.plot(scaler.inverse_transform(dataset))
plt.plot(trainPredictPlot)
plt.plot(testPredictPlot)
plt.show()

<span> The model has an average error of 22.88 i.e., 23 passenges (in thousands) on training dataset </span>

<span> The model has an average error of 49.62 i.e., 50 passenges (in thousands) on test dataset </span>